In [75]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Model
import pathlib

# Data Preprocessing

In [3]:
# # Mount to gdrive
# drive.mount('/content/gdrive')
# %cd /content/gdrive

os.chdir("../")
base_dir = os.getcwd()
print(base_dir)
data_path = "data"

d:\document\target\2021-2022\event\bootcamp\bangkit\capstone\data_organize


In [4]:
# Check directory
train_path = os.path.join(data_path, "train")
val_path = os.path.join(data_path, "validation")

# Directory with training normal/tuberculosis pictures
train_normal_dir = os.path.join(train_path, "normal")
train_tuberculosis_dir = os.path.join(train_path, "tuberculosis")
# print('total training normal images :', len(os.listdir(train_normal_dir)))
# print('total training tuberculosis images :', len(os.listdir(train_tuberculosis_dir)))


# Directory with training normal/tuberculosis pictures
validation_normal_dir = os.path.join(val_path, 'normal')
validation_tuberculosis_dir = os.path.join(val_path, 'tuberculosis')
# print('total validation normal images :', len(os.listdir( validation_normal_dir)))
# print('total validation tuberculosis images :', len(os.listdir( validation_tuberculosis_dir )))

In [5]:
# Melakukan Normalisasi
train_datagen= ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale= 1/255)

# Melakukan Instantiates train data generator
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(227, 227),
        batch_size=32,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        'data/validation',
        target_size=(227, 227),
        batch_size=32,
        class_mode='binary'
)

Found 4480 images belonging to 2 classes.
Found 1120 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'normal': 0, 'tuberculosis': 1}

# Build Model & Convert

In [ ]:
# Download the pre-trained weights
# ! curl https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [88]:
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [96]:
# Melakukan instantiates InceptionV3
pre_trained_model = InceptionV3(input_shape = (227, 227, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

# Menonaktifkan trainable weights pada layers
# untuk menjaga weight tidak berubah
for layer in pre_trained_model.layers:
  layer.trainable = False

In [101]:
# Menentapkan batas layer yang akan digunakan pada Inception
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [106]:
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = Model(pre_trained_model.input, x)
# model.summary()

In [107]:
model.compile(optimizer = SGD(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [108]:
history = model.fit(
            train_generator,
            steps_per_epoch = train_generator.samples//train_generator.batch_size,
            validation_data = validation_generator,
            validation_steps = validation_generator.samples//validation_generator.batch_size,
            epochs = 15,
            verbose = 2)

Epoch 1/15
140/140 - 565s - loss: 0.6537 - accuracy: 0.6692 - val_loss: 0.5838 - val_accuracy: 0.8196 - 565s/epoch - 4s/step
Epoch 2/15
140/140 - 481s - loss: 0.5286 - accuracy: 0.8687 - val_loss: 0.4815 - val_accuracy: 0.8616 - 481s/epoch - 3s/step
Epoch 3/15
140/140 - 484s - loss: 0.4391 - accuracy: 0.8837 - val_loss: 0.4117 - val_accuracy: 0.8714 - 484s/epoch - 3s/step
Epoch 4/15
140/140 - 485s - loss: 0.3792 - accuracy: 0.8929 - val_loss: 0.3651 - val_accuracy: 0.8875 - 485s/epoch - 3s/step
Epoch 5/15
140/140 - 525s - loss: 0.3374 - accuracy: 0.9007 - val_loss: 0.3316 - val_accuracy: 0.8920 - 525s/epoch - 4s/step
Epoch 6/15
140/140 - 524s - loss: 0.3071 - accuracy: 0.9069 - val_loss: 0.3063 - val_accuracy: 0.8991 - 524s/epoch - 4s/step
Epoch 7/15
140/140 - 546s - loss: 0.2838 - accuracy: 0.9132 - val_loss: 0.2861 - val_accuracy: 0.9000 - 546s/epoch - 4s/step
Epoch 8/15
140/140 - 489s - loss: 0.2651 - accuracy: 0.9190 - val_loss: 0.2696 - val_accuracy: 0.9080 - 489s/epoch - 3s/step


In [ ]:
saved_model = "saved_model"
tf.saved_model.save(model, saved_model)

In [110]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [111]:
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

65832280